In [ ]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4
%pip install boto3
%pip install logging

In [11]:
import os
import json
import requests
import logging
import time
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# 헤더 설정 (크롤링 차단 방지)
header = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                   "(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"),
}

# 크롤링할 네이버 뉴스 카테고리
categories = {
    "정치": "https://news.naver.com/section/100",
    "경제": "https://news.naver.com/section/101",
    "사회": "https://news.naver.com/section/102",
}

# 현재 시간 (UTC+9 기준)
CURRENT_TIME = datetime.now()

# 📰 개별 기사 크롤링 함수
def scrape_article(article_url):
    try:
        time.sleep(1)  # 요청 간격 조정
        response = requests.get(article_url, headers=header)
        soup = BeautifulSoup(response.text, "html.parser")

        # 기사 제목 찾기
        title_tag = soup.find("h2", {"id": "title_area"})
        title = title_tag.text.strip() if title_tag else "제목 없음"

        # 기사 본문 찾기
        content_tag = soup.find("article", {"id": "dic_area"})
        content = content_tag.text.strip().replace('\\"', '"').replace("\n", " ") if content_tag else "본문 없음"

        # 기사 생성 시간 찾기
        timestamp_tag = soup.find("span", {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}) 
        if timestamp_tag and "data-date-time" in timestamp_tag.attrs:
            article_time = datetime.strptime(timestamp_tag["data-date-time"], "%Y-%m-%d %H:%M:%S")
        else:
            article_time = None

        return {
            "title": title,
            "content": content,
            "timestamp": article_time,
            "link": article_url
        }
    except Exception as e:
        logging.error(f"기사 크롤링 실패: {article_url}, 오류: {e}")
        return None

# ⏳ 3시간 이내 기사 필터링 함수
def filter_recent_articles(article_urls):
    valid_articles = []
    three_hours_ago = CURRENT_TIME - timedelta(hours=3)

    for url in article_urls:
        article_data = scrape_article(url)
        print(article_data)

        if not article_data or not article_data["timestamp"]:
            continue

        # 3시간 이내 기사만 추가
        if article_data["timestamp"] >= three_hours_ago:
            article_data["timestamp"] = article_data["timestamp"].strftime("%Y-%m-%dT%H:%M:%S+09:00")
            valid_articles.append(article_data)
    return valid_articles

# 📌 헤드라인 뉴스 크롤링 함수
def check_for_new_articles(base_url, category):
    try:
        response = requests.get(base_url, headers=header)
        soup = BeautifulSoup(response.text, "html.parser")

        news_data = []
        headline_list = soup.find("ul", class_="sa_list")
        
        if not headline_list:
            logging.warning(f"{category}: 헤드라인 뉴스 리스트를 찾을 수 없음")
            return

        for item in headline_list.find_all("li", class_="sa_item _SECTION_HEADLINE")[:5]:
            title_tag = item.find("a", class_="sa_text_title")
            if not title_tag:
                continue

            title = title_tag.find("strong", class_="sa_text_strong").text.strip()
            link = title_tag["href"]

            # 상대 URL인 경우 절대 경로로 변환
            if link.startswith("/"):
                link = f"https://news.naver.com{link}"

            news_data.append({'title': title, 'link': link})

        # 🔥 3시간 이내 기사 필터링
        article_links = [news["link"] for news in news_data]
        recent_articles = filter_recent_articles(article_links)

        # 결과 출력
        for article in recent_articles:
            post_article_to_server(article)
            # print(json.dumps(article, ensure_ascii=False, indent=4))

    except Exception as e:
        logging.error(f"{category} 크롤링 실패: {e}")

def post_article_to_server(article_data):
    headers = {"Content-Type": "application/json"}
    
    try:
        logging.info(f"📤 BiteNews에 기사 전송 중: {article_data['title']}")
        response = requests.post("http://127.0.0.1:8080/ai/video", json=article_data, headers=headers)
        if response.status_code == 201:
            response_data = response.json()
            logging.info(f"📥 FastAPI 응답 수신 완료: {response_data}")

            # ✅ 숏폼 생성 완료 or 실패 로그 남김
            if response_data.get("status") == "success":
                logging.info(f"✅ 숏폼 생성 완료: {article_data['title']}")
            else:
                logging.warning(f"⚠️ 숏폼 생성 실패: {article_data['title']}, 이유: {response_data.get('message', '알 수 없음')}")

        else:
            logging.error(f"❌ BiteNews 오류 {response.status_code}: {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"❌ BiteNews 서버 연결 실패: {e}")

# 🔄 전체 카테고리 크롤링 실행 함수
def scrap_all_categories():
    for category, base_url in categories.items():
        logging.info(f"📡 {category} 크롤링 시작 ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(1)  # 요청 간격 조정

# 크롤링 실행
scrap_all_categories()


INFO:root:📡 정치 크롤링 시작 (https://news.naver.com/section/100)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): news.naver.com:443
DEBUG:urllib3.connectionpool:https://news.naver.com:443 "GET /section/100 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): n.news.naver.com:443
DEBUG:urllib3.connectionpool:https://n.news.naver.com:443 "GET /mnews/article/056/0011901161 HTTP/1.1" 200 None


{'title': '권영세, 민주당 상법 개정안 “경제 정치화…이재명식 양두구육”', 'content': "국민의힘은 야당이 국회 본회의에서 주주에 대한 이사의 충실 의무 등을 골자로 하는 상법 개정안 처리를 강행하려는 것에 “눈앞의 권력에 눈멀어 경제의 정치화를 조장하고 있다”며 “이재명식 양두구육”이라고 비판했습니다. 국민의힘 권영세 비대위원장은 오늘(27일) 국회에서 열린 비상대책위원회의에서 “상법 개정안은 기업 경영권을 위협하고 경쟁력을 떨어뜨리는 위험천만한 법안”이라며 이같이 밝혔습니다. 권 위원장은 “상장기업 다수를 차지하는 중소중견기업은 소송이 두려워 신산업 진출 등 과감한 투자 결정을 못 하게 될 거고 대한민국에서 기업가 정신은 사라질 것”이라고 지적했습니다. 또 이재명 대표를 향해 “최근 실용주의와 친기업 정책을 표방하지만, 결국 ‘이재명식 양두구육’”이라며 “양 키우겠다면서, 잡아먹을 궁리만 하는 것”이라고 비난했습니다. 그러면서, “민주당은 공직자 탄핵을 남발하며 진보 진영 변호사에게 국민 세금으로 일감을 몰아준 바 있다”면서, “상법 개정으로 인한 소송 남발 사태에 최고 수혜자가 누구인지 법안 개정 의도를 의심하지 않을 수 없다”고 주장했습니다. 권 위원장은 “한 개인의 정치적 목적을 위해 나라 경제정책이 이용당하는 일이 더 이상 발생해선 안 된다”며 “국민의힘은 자유시장경제 가치를 실현할 것”이라고 강조했습니다.[사진 출처 : 연합뉴스]■ 제보하기▷ 전화 : 02-781-1234, 4444▷ 이메일 : kbs1234@kbs.co.kr▷ 카카오톡 : 'KBS제보' 검색, 채널 추가▷ 네이버, 유튜브에서 KBS뉴스를 구독해주세요!", 'timestamp': datetime.datetime(2025, 2, 27, 9, 24, 32), 'link': 'https://n.news.naver.com/mnews/article/056/0011901161'}


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): n.news.naver.com:443
DEBUG:urllib3.connectionpool:https://n.news.naver.com:443 "GET /mnews/article/421/0008100804 HTTP/1.1" 200 None


{'title': '권성동 "이재명 정치노선 좌클릭도 우클릭도 아닌 罪클릭"', 'content': '"양치기소년도 무릎꿇고 기절할 거짓말 덮어쓰기""명태균특검법은 명태균-민주당 공모 정치 공작"    권성동 국민의힘 원내대표가 26일 오전 서울 여의도 국회에서 열린 주주 권익 및 기업 가치 제고를 위한 경제단체 간담회에서 발언하고 있다. 2025.2.26/뉴스1 ⓒ News1 이광호 기자(서울=뉴스1) 한상희 손승환 기자 = 권성동 국민의힘 원내대표는 27일 이재명 더불어민주당의 공직선거법 위반 사건과 관련해 "이 대표의 정치노선은 좌클릭도 아니고 우클릭도 아니다. 오직 \'죄(罪)클릭"만 있을 뿐"이라고 맹비난했다.권 원내대표는 이날 오전 서울 여의도 국회에서 열린 비상대책위원회의에서 "이 대표는 8개 사건에서 12개 혐의로 5곳의 재판부에서 재판을 받고있다"며 이같이 말했다.그는 "이 대표는 공직선거법상의 허위사실공표죄를 두고 \'전 세계에서 대한민국이 유일하다\'라고 또 거짓말을 했다"며 "양치기 소년도 무릎을 꿇고 기절할 만큼의 거짓말 덮어쓰기"라고 주장했다. 고 김문기 전 성남도시개발공사 개발1처장을 몰랐다고 했던 이 대표의 발언에 대해서도 "차라리 대학 시절 고시반에서 함께 공부한 저 권성동도 모르는 사람이라고 하라"고 비꼬았다. 권 원내대표는 이 대표가 전날 법원 출석 당시 \'세상의 이치라고 하는 게 다 상식과 원칙대로 가게 돼 있다\'고 말한 것을 언급하며 "당연하다. 그래서 이 대표는 그 세상의 이치에 따라 법의 심판을 받게 될 것"이라고 경고했다. 민주당이 이날 국회 본회의에서 처리할 예정인 \'명태균 특검법\'에 대해서는 "(정치 브로커) 명태균과 민주당이 공모한 정치 공작"이라고 주장했다. 권 원내대표는 "한낱 선거 브로커가 하는 허황된 말들을 신의 말씀처럼 떠받들면서 특검을 도입해서 여당과 보수진영을 무차별적으로 초토화하겠다는 것"이라며 "조기 대선을 겨냥해 \'제2의 김대업\'을 만들겠다는 정략 특검"이라고 비판했다. 이어 "명태균

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): n.news.naver.com:443
DEBUG:urllib3.connectionpool:https://n.news.naver.com:443 "GET /mnews/article/437/0000431468 HTTP/1.1" 200 None


...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): n.news.naver.com:443
DEBUG:urllib3.connectionpool:https://n.news.naver.com:443 "GET /mnews/article/023/0003890397 HTTP/1.1" 200 None


{'title': '법원, 이재명 공직선거법 2심 다음 달 26일 선고', 'content': '檢, 징역 2년 구형…1심은 징역 1년·집행유예 2년    2025년 2월 26일 오후 서울 서초구 서울중앙지방법원에서 이재명 대표가 공판에 출석하기 위해 청사로 향하고 있다. / 장련성 기자\t\t\t\t\t\t\t\t\t\t이재명 더불어민주당 대표의 공직선거법 위반 사건 항소심 선고가 오는 3월 26일 오후 2시에 열린다.26일 서울고법 형사 6-2부(재판장 최은정)는 이 대표 항소심 결심 공판을 열고 다음 달 26일을 선고 기일로 잡았다. 검찰은 1심과 같은 징역 2년을 구형했다. 이 대표는 “구형이 중요한 게 아니라 실체가 중요하다. 사법부가 현명하게 그리고 정의롭게 실체적 진실에 입각해서 잘 판단할 것으로 생각한다”고 말했다.◇檢 “대통령 당선을 위한 반복적 거짓말”이 대표는 지난 대선을 앞둔 2021년 12월 방송 4곳에서 대장동 개발 실무자인 고(故) 김문기 전 성남도시개발공사 처장을 성남시장 재임 기간에 알았으면서도 몰랐다고 하고, 2021년 10월 국정감사에 나와 “국토교통부 협박으로 백현동 개발 부지 용도를 4단계 상향 조정했다”고 거짓말한 혐의(공직선거법 위반)로 2022년 9월 불구속 기소됐다.작년 11월 15일 1심은 이 대표에게 징역 1년에 집행유예 2년을 선고했다. “2015년 호주 출장 중 김 전 처장과 골프를 치지 않았다”와 백현동 부지 용도 변경과 관련한 이 대표 발언은 유죄를 인정했지만, “김 전 처장을 성남시장 재직 시절에는 몰랐고 경기지사가 된 후 알았다”는 말은 무죄라고 봤다.이날 검찰은 이 대표에게 징역 2년을 선고해달라고 재판부에 요청하면서 이 대표가 김 전 처장을 모른다고 한 부분에 대해 “김 전 처장과의 골프는 외유성이라는 논란을 피하기 위해 사전에 은밀히 준비됐고, 시간은 유동규까지 전 성남도시개발공사 기획본부장까지 3명이 5시간이었다”고 반박했다. 이 대표가 이틀 후 김 전 처장과 낚시를 함께한 점도 강조하며 “소수

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): n.news.naver.com:443
DEBUG:urllib3.connectionpool:https://n.news.naver.com:443 "GET /mnews/article/277/0005552668 HTTP/1.1" 200 None
INFO:root:📤 BiteNews에 기사 전송 중: 울산시, 블루수소 충전 인프라 구축 투자유치
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8080


{'title': '"하늘이 도왔다" 교량 붕괴 30ｍ 추락에도 경상 그친 생존자', 'content': '추락 사고 직후 가족에 생존담 전달산비탈 흙더미로 떨어져 충격 완화 추정서울세종고속도로 청룡천교 붕괴 사고로 10명의 사상자가 발생한 가운데, 사고 현장에서 수십 미터 아래로 추락한 60대가 극적으로 생존한 것으로 알려졌다. 대형 사고에도 비교적 몸 상태가 심각하지 않은 것으로 알려진 그는 가족에게 "하늘이 도왔다"고 말한 것으로 전해졌다.    25일 경기 안성시 서운면 산평리 소재 서울세종고속도로 천안~안성구간 9공구 천용천교 건설 현장에서 교량 연결작업 중 교각에 올려놓았던 상판이 무너지는 사고가 발생했다. 사진은 사고 현장 모습. 연합뉴스26일 연합뉴스 보도에 따르면 중국 국적 근로자 A(62)씨는 전날 경기 안성시 서운면 산평리 서울세종고속도로 천안~안성 구간 9공구 청룡천교 건설 현장에서 콘크리트 상판 구조물과 함께 추락했다. 그는 사고 초기 얼굴 등에 찰과상과 타박상을 입은 경상자로 분류됐다가, 안와골절 등으로 이날 화성시 한 대학병원에서 수술받고 회복 중이다.A씨는 현재 거동이 불편한 상태지만, 대형 사고를 당했는데도 비교적 몸 상태가 심각하지 않은 것으로 알려졌다. 사고 직후 A씨는 가족에게 "30m 아래로 떨어진 것 같다"며 "하늘이 도왔다"고 말한 것으로 전해졌다. 반면 A씨를 제외한 다른 근로자 5명은 크게 다쳐 중환자실 등에서 치료받고 있다.사고 당시 A씨는 교량 상판 구조물인 거더(대들보 기능의 구조물)에 설치된 런처(거더를 인양·설치하는 대형 장비)를 후방으로 이동시키는 작업 중이었던 것으로 파악됐다. A씨가 어느 지점에 있었는지는 정확히 조사되지 않았으나, 런처 아래 부근에 있다가 거더가 V자 모양으로 붕괴하면서 같이 추락한 것으로 전해졌다.사고가 발생한 교량 높이는 최고 52m다. 다만 A씨는 추락 지점과 가까운 산비탈 아래 흙더미로 떨어지면서 다른 작업자들보다 상대적으로 작은 충격을 받았을 것으로 추정된다.이번 사고는 25

DEBUG:urllib3.connectionpool:http://127.0.0.1:8080 "POST /ai/video HTTP/1.1" 201 65
INFO:root:📥 FastAPI 응답 수신 완료: {'status': 'success', 'message': '✅ 기사 숏폼 생성 성공!'}
INFO:root:✅ 숏폼 생성 완료: 울산시, 블루수소 충전 인프라 구축 투자유치
